In [55]:
import pandas as pd
import numpy as np

In [56]:
# scenario 1:
# NaN -> drop
# A
# NaN

# scenario 2:
# A
# NaN -> assign A
# B
# NaN -> assign B

df = pd.DataFrame({
    "col1":["2022-01-03","2022-01-05","2022-01-02", "2022-01-01","2022-01-06", "2022-01-04", "2022-01-03"],
    "col2":["col2row1","col2row2",np.nan, np.nan, np.nan, "col2row6", np.nan],
    "col3": ['admin1','admin1','admin2','admin1','admin2','admin2', "admin2"]
})

df.sort_values(['col1'])

,col1,col2,col3
3,2022-01-01,NaN,admin1
2,2022-01-02,NaN,admin2
0,2022-01-03,col2row1,admin1
6,2022-01-03,NaN,admin2
5,2022-01-04,col2row6,admin2
1,2022-01-05,col2row2,admin1
4,2022-01-06,NaN,admin2


In [57]:
df2 = df.sort_values(['col3','col1']).copy()

display(df2)

def findFirstInvalidLabelIndex(admin: str):
    index = df2.loc[df2['col3'] == admin, 'col2'].isna().idxmax()
    # A -> findFirstInvalidLabelIndex() and firstFirstValidLabelIndex()
    # B
    if index == findFirstValidLabelIndex(admin):
        return -1
    return index

# print(findFirstInvalidLabelIndex('admin1'))

def findFirstValidLabelIndex(admin: str) -> int | None:
    return df2.loc[df2['col3'] == admin, 'col2'].first_valid_index()

# print(findFirstValidLabelIndex('admin1'))

def findLastValidLabelIndex(admin: str) -> int | None:
    return df2.loc[df2['col3'] == admin, 'col2'].last_valid_index()

# print(findLastValidLabelIndex('admin1'))

def replaceFirstInvalidIndexWithValidIndex(admin: str):
    firstInvalidLabelIndex = findFirstInvalidLabelIndex(admin)
    if firstInvalidLabelIndex == -1:
        return

    firstValidLabelIndex = findFirstValidLabelIndex(admin)
    if firstValidLabelIndex == None:
        return

    lastValidLabelIndex = findLastValidLabelIndex(admin)
    if lastValidLabelIndex == None:
        return
    
    global df2
    # scenario 1:
    # NaN -> drop
    # A
    # NaN
    if df2.index.get_loc(firstInvalidLabelIndex) < \
            df2.index.get_loc(firstValidLabelIndex):
        df2 = df2.drop(labels=firstInvalidLabelIndex)
        return
    # scenario 2:
    # A
    # NaN -> assign A
    # B
    # NaN -> assign B
    if df2.index.get_loc(firstInvalidLabelIndex) > \
            df2.index.get_loc(firstValidLabelIndex):
        # scenario 3:
        # A
        # A
        # B   -> lastValidLabelIndex
        # NaN -> firstInvalidLabelIndex
        if df2.index.get_loc(lastValidLabelIndex) != 0:
            df2.loc[firstInvalidLabelIndex, 'col2'] = df2.loc[lastValidLabelIndex, 'col2']
            return
        # scenario 2: ...
        df2.loc[firstInvalidLabelIndex, 'col2'] = df2.loc[firstValidLabelIndex, 'col2']
        return

# print('admin1', findFirstInvalidLabelIndex('admin1'))

# replaceFirstInvalidIndexWithValidIndex('admin1')

# display(df2)

# print('admin1', findFirstInvalidLabelIndex('admin1'))

# print('admin2', findFirstInvalidLabelIndex('admin2'))

# replaceFirstInvalidIndexWithValidIndex('admin2')

# display(df2)

# print('admin2', findFirstInvalidLabelIndex('admin2'))

# replaceFirstInvalidIndexWithValidIndex('admin2')

# display(df2)

# print('admin2 problem', findFirstInvalidLabelIndex('admin2'))

for admin in ['admin1', 'admin2']:
    while findFirstInvalidLabelIndex(admin) != -1:
        replaceFirstInvalidIndexWithValidIndex(admin)

display(df2)

,col1,col2,col3
3,2022-01-01,NaN,admin1
0,2022-01-03,col2row1,admin1
1,2022-01-05,col2row2,admin1
2,2022-01-02,NaN,admin2
6,2022-01-03,NaN,admin2
5,2022-01-04,col2row6,admin2
4,2022-01-06,NaN,admin2


,col1,col2,col3
0,2022-01-03,col2row1,admin1
1,2022-01-05,col2row2,admin1
5,2022-01-04,col2row6,admin2
4,2022-01-06,col2row6,admin2


In [61]:
temp_df = pd.DataFrame({'admin2': ['admin1','admin2','admin3','admin4'], 'fips': ['fips1','fips2',np.nan,'fips4'], 'location_geom': ['location1', 'location2', 'location3', np.nan], 'confirmed': [5,6,7,8]})
display(temp_df.groupby(['admin2','fips','location_geom']).agg(['min','max']))

confirmed    
                                 min max
admin2 fips  location_geom              
admin1 fips1 location1             5   5
admin2 fips2 location2             6   6